# Who uses pandas?
# Who uses numpy?

## Blaze - A Quick Tour

Blaze provides a lightweight interface on top of pre-existing computational infrastructure.  This notebook gives a quick overview of how Blaze interacts with a variety of data types.

In [ ]:
%reload_ext autotime

In [ ]:
from blaze import Data, by, compute

### Blaze wraps pre-existing data

Blaze interacts with normal Python objects.  Operations on Blaze `Data` objects create expression trees.  

These expressions deliver an intuitive numpy/pandas-like feel.

### Lists

Starting small, Blaze interacts happily with collections of data.  

It uses Pandas for pretty notebook printing.

In [131]:
x = Data([1, 2, 3, 4, 5])
x

,None
0,1
1,2
2,3
3,4
4,5


time: 12 ms


In [132]:
x[x > 2] * 10

,None
0,30
1,40
2,50


time: 17.4 ms


In [134]:
_132.dshape

dshape("var * int64")

time: 1.77 ms


In [133]:
x.dshape

dshape("5 * int64")

time: 1.72 ms


## Or Tabular, Pandas-like datasets

Slightly more exciting, Blaze operates on tabular data

In [135]:
L = [[1, 'Alice',   100],
     [2, 'Bob',    -200],
     [3, 'Charlie', 300],
     [4, 'Dennis',  400],
     [5, 'Edith',  -500]]

time: 1.6 ms


In [136]:
x = Data(L, fields=['id', 'name', 'amount'])

time: 1.44 ms


In [137]:
x.dshape

dshape("5 * {id: int64, name: string, amount: int64}")

time: 1.72 ms


In [138]:
x

,id,name,amount
0,1,Alice,100
1,2,Bob,-200
2,3,Charlie,300
3,4,Dennis,400
4,5,Edith,-500


time: 9.66 ms


In [139]:
deadbeats = x[x.amount < 0].name
deadbeats

,name
0,Bob
1,Edith


time: 13.6 ms


## Or it can even just drive pandas

Blaze doesn't do work, it just tells other systems to do work.

In the previous example, Blaze told Python which for-loops to write.  In this example, it calls the right functions in Pandas.  

The user experience is identical, only performance differs.

In [140]:
from pandas import DataFrame

df = DataFrame([[1, 'Alice',   100],                         
                [2, 'Bob',    -200],
                [3, 'Charlie', 300],
                [4, 'Denis',   400],
                [5, 'Edith',  -500]], columns=['id', 'name', 'amount'])

time: 3.12 ms


In [141]:
df

,id,name,amount
0,1,Alice,100
1,2,Bob,-200
2,3,Charlie,300
3,4,Denis,400
4,5,Edith,-500


time: 5.65 ms


In [142]:
x = Data(df)
x

,id,name,amount
0,1,Alice,100
1,2,Bob,-200
2,3,Charlie,300
3,4,Denis,400
4,5,Edith,-500


time: 9.49 ms


In [143]:
deadbeats = x[x.amount < 0].name
deadbeats

,name
1,Bob
4,Edith


time: 16.4 ms


Calling `compute`, we see that Blaze returns a thing like what it was given.

In [144]:
type(deadbeats)

blaze.expr.expressions.Field

time: 1.51 ms


In [145]:
compute(deadbeats)

1      Bob
4    Edith
Name: name, dtype: object

time: 4.97 ms


In [146]:
type(compute(deadbeats))

pandas.core.series.Series

time: 4.36 ms


### Other data types like SQLAlchemy Tables

Blaze extends beyond just Python and Pandas (that's the main motivation.)  

Here it drives SQLAlchemy.

In [149]:
from sqlalchemy import Table, Column, MetaData, Integer, String, create_engine

tab = Table('bank', MetaData(),
            Column('id', Integer),
            Column('name', String),
            Column('amount', Integer))

time: 2.26 ms


In [150]:
x = Data(tab)
x.dshape

dshape("var * {id: ?int32, name: ?string, amount: ?int32}")

time: 2.21 ms


Just like computations on pandas objects produce pandas objects, computations on SQLAlchemy tables produce SQLAlchemy Select statements.  

In [151]:
deadbeats = x[x.amount < 0].name
compute(deadbeats)

<sqlalchemy.sql.selectable.Select at 0x1194454a8; Select object>

time: 5.94 ms


In [153]:
print(compute(deadbeats))  # SQLAlchemy generates SQL

SELECT bank.name 
FROM bank 
WHERE bank.amount < :amount_1
time: 3.55 ms


### Connect to a real database

When we drive a SQLAlchemy table connected to a database we get actual computation.

In [154]:
engine = create_engine('sqlite:///../blaze/blaze/examples/data/iris.db')

time: 2.31 ms


In [155]:
x = Data(engine)
x

Data:       Engine(sqlite:///../blaze/blaze/examples/data/iris.db)
DataShape:  {
  iris: var * {
    sepal_length: ?float64,
    sepal_width: ?float64,
    petal_length: ?float64,
    petal_width: ?float64,
    species: ?string
  ...

time: 8.49 ms


In [157]:
x.fields

['iris']

time: 1.59 ms


In [159]:
x.iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


time: 19 ms


In [160]:
by(
    x.iris.species,
    shortest=x.iris.sepal_length.min(),
    longest=x.iris.sepal_length.max()
)

,species,longest,shortest
0,Iris-setosa,5.8,4.3
1,Iris-versicolor,7.0,4.9
2,Iris-virginica,7.9,4.9


time: 48.7 ms


### Use URI strings to ease access

Often just figuring out how to produce the relevant Python object can be a challenge.

Blaze supports many formats of URI strings

In [161]:
x = Data('sqlite:///../blaze/blaze/examples/data/iris.db::iris')

time: 5.35 ms


In [162]:
x

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


time: 16.8 ms


### Once you have SQL, you might as well go big

In [163]:
x = Data('redshift://cio@localhost:15439/dev', connect_args=dict(sslmode='verify-ca'))

time: 1.05 s


In [164]:
x.fields

['boros', 'trip']

time: 1.63 ms


In [165]:
x.trip

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tolls_amount,tip_amount,total_amount,mta_tax,fare_amount,payment_type,surcharge
0,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:54:15,2013-01-07 23:58:20,2,244,0.7,-73.974602,40.759945,-73.984734,40.759388,0.0,0,6.0,0.5,5.0,CSH,0.5
1,0B57B9633A2FECD3D3B1944AFC7471CF,CCD4367B417ED6634D986F573A552A62,CMT,1,N,2013-01-07 12:39:18,2013-01-07 13:10:56,3,1898,10.7,-73.989937,40.756775,-73.865250,40.770630,4.8,0,39.3,0.5,34.0,CSH,0.0
2,E12F6AF991172EAC3553144A0AF75A19,06918214E951FA0003D1CC54955C2AB0,CMT,1,N,2013-01-08 09:50:05,2013-01-08 10:02:54,1,768,0.7,-73.995560,40.749294,-73.988686,40.759052,0.0,0,9.5,0.5,9.0,CSH,0.0
3,3349F919AA8AE5DC9C50A3773EA45BD8,7CE849FEF67514F080AF80D990F7EF7F,CMT,1,N,2013-01-10 14:27:28,2013-01-10 14:45:21,1,1073,4.4,-74.010391,40.708702,-73.987846,40.756104,0.0,0,17.0,0.5,16.5,CSH,0.0
4,0C5296F3C8B16E702F8F2E06F5106552,D2363240A9295EF570FC6069BC4F4C92,CMT,1,N,2013-01-07 22:25:46,2013-01-07 22:36:56,1,669,2.3,-73.989708,40.756714,-73.977615,40.787575,0.0,0,11.5,0.5,10.5,CSH,0.5
5,4C005EEBAA7BF26B84B21586332488A2,351BE7D984BE17DB2FA80A748E816472,CMT,1,N,2013-01-07 21:13:02,2013-01-07 21:22:31,1,568,1.1,-73.978439,40.764679,-73.977684,40.777004,0.0,0,9.0,0.5,8.0,CSH,0.5
6,A3B17384165197E18CA0A1BB61277EE9,B8396B62883EA332EFD2771A6B031D05,CMT,1,N,2013-01-07 06:26:32,2013-01-07 06:28:24,2,111,0.5,-73.965317,40.769375,-73.967133,40.763699,0.0,0,5.0,0.5,4.0,CSH,0.5
7,0F9E0728AB1E40D5CEB0C6EDBF805CCB,8434E8A33D8C0150573FAA00B8A9ABF5,CMT,1,N,2013-01-05 19:04:43,2013-01-05 19:13:58,1,555,2.8,-73.966682,40.761139,-73.938515,40.792332,0.0,0,10.5,0.5,10.0,CSH,0.0
8,2C159C8FCCDE50174CF6CFC07E75F1BA,51C567531541EE45C5EE86E956E46E4C,CMT,1,N,2013-01-07 04:45:14,2013-01-07 04:48:56,1,222,0.9,-73.991936,40.749622,-73.978531,40.751308,0.0,0,6.5,0.5,5.5,CSH,0.5
9,DD98E2C3AF5C47B4449F720ECC5778D4,79807332B275653A2473554C7328500A,CMT,1,N,2013-01-02 06:58:08,2013-01-02 07:24:24,1,1576,3.0,-73.987022,40.759373,-73.862022,40.768017,4.8,0,7.8,0.5,2.5,CSH,0.0


time: 193 ms


In [166]:
x.boros

,lon,lat,name
0,-74.0505,40.5664,Staten Island
1,-74.0501,40.5664,Staten Island
2,-74.0493,40.5659,Staten Island
3,-74.0500,40.5653,Staten Island
4,-74.0507,40.5660,Staten Island
5,-74.0509,40.5661,Staten Island
6,-74.0508,40.5662,Staten Island
7,-74.0507,40.5663,Staten Island
8,-74.0508,40.5664,Staten Island
9,-74.0509,40.5665,Staten Island


time: 126 ms


In [167]:
by(
    x.trip.passenger_count,
    avg_fare_amount=x.trip.total_amount.mean()
).sort('passenger_count', ascending=False)

,passenger_count,avg_fare_amount
0,NaN,25.419503
1,9,29.372308
2,8,29.805200
3,7,35.472857
4,6,14.755711
5,5,14.947678
6,4,15.069749
7,3,14.934853
8,2,15.505316
9,1,14.623773


time: 12.4 s


In [168]:
x = Data('impala://54.234.194.112/default::reddit_parquet')

time: 1.83 s


In [170]:
values = x.ups.count_values()

time: 1.54 ms


In [171]:
nrows = int(values['count'].sum())

time: 1min 31s


In [172]:
values

,ups,count
0,1,728318693
1,2,300349896
2,3,139258338
3,0,83314693
4,4,53445185
5,5,48616973
6,6,35632934
7,-1,28568202
8,7,25990637
9,8,19789446


time: 2min 33s


In [ ]:
nrows